# Trabajo práctico 9 - Calibración

**Alumnos:**

- Carol lugones Ignacio (100073)
- Torresetti Lisandro (99846)

## Objetivo

En base a las funciones de calibración implementadas en OpenCV realizar la calibración de sus celulares y devolver la matriz de parámetros intrinsecos.

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import PIL.ExifTags
import PIL.Image
import pprint
%matplotlib inline

In [ ]:
def plotter(image, title = '', imgSize = (18,9), grayScale = False, step = 100): #Funcion auxiliar para realizar los graficos
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.yticks(np.arange(0, len(image), step))
    plt.xticks(np.arange(0, len(image[0]), step), rotation=90)
    plt.show() 

El patrón a analizar es el siguiente:

In [ ]:
pattern = cv.imread('ChessBoardPattern.png')
plotter(pattern, 'Chess Board Pattern')

Ahora definimos el tamaño del tablero (la cantidad de puntos es igual a la cantidad de puntos en los que un cuadrado negro toca a uno blanco), y la lista de puntos a reconocer, tomando como origen (0, 0, 0) la esquina que se encuentra en las coordenadas (100, 100).

In [ ]:
chessBoardSize  = (7, 7)
objp = np.zeros((np.prod(chessBoardSize), 3),  dtype=np.float32)
objp[:, :2] = np.mgrid[0:chessBoardSize[0], 0:chessBoardSize[1]].T.reshape(-1, 2)

A continuación cargamos las imágenes tomadas al patrón de distintos ángulos.

In [ ]:
img_fnames = glob('./fotos/*')
imgsGray = []
imgsColor = []
for imgName in img_fnames:
    img = cv.imread(imgName)
    imgsColor.append(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    imgsGray.append(cv.cvtColor(img, cv.COLOR_BGR2GRAY))

for img in imgsColor: #Imprimimos las imagenes tomadas del patron
    plotter(img)

Ahora definimos dos funciones, una para encontrar las esquinas y otra para calibrar la cámara que será usada más adelante.

In [ ]:
def findCorners(imgsColor, imgsGray, plot=True, maxCount = 25, epsilon = 0.001, flag=cv.CALIB_CB_ADAPTIVE_THRESH):
    imgPoints = []
    objPoints = []
    criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_MAX_ITER, maxCount, epsilon)
    cb_flags = flag 
    for imgColor, imgGray in zip(imgsColor, imgsGray):
        imgColor = imgColor.copy()
        ret, corners = cv.findChessboardCorners(imgGray, chessBoardSize, flags=cb_flags)
        if ret:
            objPoints.append(objp)
            corners_subp = cv.cornerSubPix(imgGray, corners, (5, 5), (-1, -1), criteria)
            imgPoints.append(corners_subp)
            cv.drawChessboardCorners(imgColor, chessBoardSize, corners_subp, ret)
            if plot:
                plotter(imgColor)
    return imgPoints, objPoints

def calibrateCamera(objPoints, imgPoints, widthAndHeight):
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objPoints, imgPoints, widthAndHeight, None, None)
    print('Camera Matrix: \n{}'.format(mtx))
    print('\nDistortion Coefficients: \n{}\n'.format(dist))

In [ ]:
imgPoints, objPoints = findCorners(imgsColor, imgsGray)
height, width = imgsGray[0].shape
calibrateCamera(objPoints, imgPoints, (width,height))

Si abrimos las imágenes anteriores en una pestaña nueva puede verse que encontró correctamente las esquinas.

In [ ]:
height, width = imgsGray[0].shape
calibrateCamera(objPoints, imgPoints, (width,height)) #este es el resultado que se pone en las conclusiones

## Variación de los flags para _findChessboardCorners_

Volveremos a realizar el proceso anterior pero esta vez variando los flags que se le pueden pasar a la función _findChessboardCorners_ de _openCV_. Los flags que se pueden utilizar son:
+ CALIB_CB_NORMALIZE_IMAGE
+ CALIB_CB_FILTER_QUADS
+ CALIB_CB_FAST_CHECK

In [ ]:
flagsChessBoard = [cv.CALIB_CB_NORMALIZE_IMAGE, cv.CALIB_CB_FILTER_QUADS, cv.CALIB_CB_FAST_CHECK]
flagsName = ['CALIB_CB_NORMALIZE_IMAGE', 'CALIB_CB_FILTER_QUADS', 'CALIB_CB_FAST_CHECK']

def tryFlags(flags):
    for flagNum, flag in enumerate(flags):
        imgPoints, objPoints = findCorners(imgsColor, imgsGray, plot=False, flag=flag)
        height, width = imgsGray[0].shape
        print('Result with {}'.format(flagsName[flagNum]))
        calibrateCamera(objPoints, imgPoints, (width,height))

In [ ]:
tryFlags(flagsChessBoard)

Se puede apreciar que variando los flags los resultados obtenidos son muy similares entre si. Y estos a su vez son similares con la primer matriz que se encontró utilizando el flag CALIB_CB_ADAPTIVE_THRESH

## Variación de las iteraciones

Ahora vamos a variar la cantidad de iteraciones, en los casos anteriores el número máximo fue de 25, ahora probaremos con 5, 50, 100 y 1000

In [ ]:
def iterations():
    for it in [5, 50, 100, 1000]:
        imgPoints, objPoints = findCorners(imgsColor, imgsGray, plot=False, maxCount=it)
        height, width = imgsGray[0].shape
        print('Result with {} iterarions'.format(it))
        calibrateCamera(objPoints, imgPoints, (width,height))
        
iterations()

Se puede apreciar que nuevamente los valores son similares, pero el que posee más diferencias con todos los demas es el caso de cuando se pone un límite de 5 iteraciones.

Por último vamos a analizar la información que posee una imagen para comparar con los resultados obtenidos

In [ ]:
def getInfo(imgNum = 0, focalLengthInfo=False):
    
    exif_img = PIL.Image.open(img_fnames[2])

    exif_data = {
     PIL.ExifTags.TAGS[k]: v
     for k, v in exif_img._getexif().items()
     if k in PIL.ExifTags.TAGS
    }
    imgName = img_fnames[imgNum].split('/')[-1]
    if focalLengthInfo:
        focal_length_exif = exif_data['FocalLength']
        focal_length = focal_length_exif[0]/focal_length_exif[1]
        print('Focal Length of the image {} = {}mm'.format(imgName, focal_length))
    else:
        print('Full Info of the image {}:'.format(imgName))
        pprint.pprint(exif_data)

In [ ]:
getInfo(focalLengthInfo=True)

También se puede obtener la siguiente información de las imágenes

In [ ]:
getInfo()

## Conclusiones

En este trabajo práctico se pudo apreciar como con funciones de openCV es sencillo obtener los parámetros intrinsecos de la cámara, como otra información.

Al variar los _flags_ y el _número de iteraciones_ no se observaron cambios grandes en los resultados de la matriz obtenida, lo más distinto que se obtuvo fue cuando se utilizaron 5 iteraciones.

El proceso de calibración es muy performante, correr este notebook es bastante rápido, siendo que se probaron con variaciones de parámetros.

Algo que llamó la atención es toda la información que se puede obtener de una foto, como la distancia focal, el número F, tiempo de exposición hasta incluso la marca del celular.

Por último, resumimos los valores obtenidos de la matriz de parámetros intrínsecos:
+ fx = 3.31104416e+03
+ fy = 3.31114074e+03
+ cx = 1.44845095e+03
+ cy = 1.91976918e+03